In [1]:
import numpy as np
import pylab
from tqdm.notebook import tqdm
import numpy as np
import math
import pandas as pd
from multiprocessing import Pool
from multiprocessing import cpu_count

In [2]:
data = pd.read_csv('/Users/weronikasadzik/Downloads/archive (4)/monthly_csv.csv', index_col=0)

# Model Blacka-Scholesa

Równanie Blacka-Scholesa:

$dX(t) = \mu X(t) dt + \sigma X(t) d W_t$

$X(t)$ - cena akcji w t-tym momencie

$t\in [0;T]$, $\mu \in \mathbb{R}$ (współczynnik dryfu), $\sigma > 0$ (współczynnik dyfuzji)

$W_t$ - proces Wienera

## Estymacja parametrów $\mu, \sigma$ metodą największej wiarygodności

In [380]:
def estimate_parameters_mle(f, g, x):
    f_min = np.inf
    a = np.linspace(0.0, 0.1, 1001)
    b = np.linspace(0.0, 1.0, 1001)
    for th1 in tqdm(a[1:]):
        for th2 in b[1:]:
            f_log = 0.0
            for j in range(1, 47):
                f_res = f(th1, x[j])
                g_res = g(th2, x[j])
                mu = x[j] + f_res
                sigma = g_res
                temp = -(x[j+1] - mu) ** 2 / (2.0 * sigma**2) - np.log(np.sqrt(2.0 * np.pi * sigma**2))
                f_log -= temp
            if f_log < f_min:
                f_min = f_log
                th1min = th1
                th2min = th2

    print(f"mu: {th1min}, sigma: {th2min}, f_min: {f_min}")

In [ ]:
x = data.values
x = [val[0] for val in x]
x = np.array([0, 18, 22, 26, 16, 19, 21, 18, 22, 25, 31, 30, 34, 31, 25, 21, 24, 21, 28, 24, 26, 32,
              33, 36, 39, 32, 33, 42, 44, 43, 48, 50, 56, 57, 59, 51, 49, 49, 57, 69, 72, 75, 76,
              78, 73, 73, 75, 86], dtype=np.float32)

f = lambda theta, x: theta * x
g = lambda theta, x: np.sqrt(theta * x)
estimate_parameters_mle(f, g, x)

## Test Goodness-of-Fit

In [379]:
import numpy as np
import math
from scipy.stats import chi2_contingency
from concurrent.futures import ThreadPoolExecutor

def simulate_data(SDE_model, x_prev, num_simulations):
    simulated_data = np.zeros(num_simulations)
    dt = 1  # Time step size
    f, g = SDE_model(x_prev)  # Function parameters of the SDE model
    
    for j in range(num_simulations):
        rand1 = np.random.normal(0, 1)
        x = x_prev + f * dt + math.sqrt(dt) * g * rand1
        simulated_data[j] = x
    
    return simulated_data

def goodness_of_fit_test(observed_data, SDE_model, num_simulations):
    # Generate simulated data based on the SDE model
    simulated_data = np.zeros_like(observed_data)
    with ThreadPoolExecutor() as executor:
        for i in range(1, len(observed_data)):
            x_prev = observed_data[i-1]
            simulated_data[i] = np.mean(list(executor.submit(simulate_data, SDE_model, x_prev, num_simulations).result()))
    
    # Calculate observed and expected frequencies
    observed_freq = np.histogram(observed_data, bins='auto')[0]
    expected_freq = np.histogram(simulated_data, bins='auto')[0]
    
    # Pad the smaller array with zeros to make both arrays have the same size
    max_size = max(len(observed_freq), len(expected_freq))
    observed_freq = np.pad(observed_freq, (0, max_size - len(observed_freq)))
    expected_freq = np.pad(expected_freq, (0, max_size - len(expected_freq)))
    
    # Perform chi-squared test
    contingency_table = np.vstack((observed_freq, expected_freq))
    chi2_stat, p_value = chi2_contingency(contingency_table)[:2]  # Take only the first two elements of the returned tuple
    
    # Determine if the model has goodness-of-fit or lack-of-fit
    threshold = 0.005  # Significance level
    if p_value < threshold:
        result = "Lack-of-fit"
    else:
        result = "Goodness-of-fit"
    
    return chi2_stat, p_value, result

# Example usage:
observed_data = np.array([18, 22, 26, 16, 19, 21, 18, 22, 25, 31, 30, 34, 31, 25, 21, 24, 21, 28, 24, 26, 32, 33, 36, 39, 32,
          33, 42, 44, 43, 48, 50, 56, 57, 59, 51, 49, 49, 57, 69, 72, 75, 76, 78, 73, 73, 75, 86])

# Define the SDE model function
def SDE_model(x):
    th1 = .0361
    th2 = 0.69
    f = th1 * x
    g = math.sqrt(th2 * x)
    return f, g

num_simulations = 10000

# Perform the goodness-of-fit test
chi2_stat, p_value, result = goodness_of_fit_test(observed_data, SDE_model, num_simulations)

print("Chi-square statistic:", chi2_stat)
print("P-value:", p_value)
print("Result:", result)

Chi-square statistic: 17.623619655198606
P-value: 0.007244955698245388
Result: Goodness-of-fit
